<a href="https://colab.research.google.com/github/lucprosa/dataeng-basic-course/blob/main/spark/challenges/challenge_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setting up PySpark

In [1]:

from pyspark.sql import DataFrame, SparkSession
from pyspark.sql.functions import *
import pyspark.sql.functions as F
from pyspark.sql.types import *

spark = SparkSession.builder.master('local').appName('Read').getOrCreate()

24/12/02 00:27:18 WARN Utils: Your hostname, Iness-MacBook-Pro.local resolves to a loopback address: 127.0.0.1; using 192.168.1.9 instead (on interface en0)
24/12/02 00:27:18 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/12/02 00:27:18 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/12/02 00:27:19 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
24/12/02 00:27:19 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.
24/12/02 00:27:19 WARN Utils: Service 'SparkUI' could not bind on port 4042. Attempting port 4043.


In [21]:
df=(spark.read.format("parquet")
.load("./content/lake/gold/vehicles_enriched/*"))


In [22]:
#Check data 
df.show(2)

+-------+--------+--------------+-------+---------+-------+---------+----------+--------+---------------------+--------+-----+-------+-------------------+--------------------+--------------------+--------------------+----------+
|bearing|block_id|current_status|     id| latitude|line_id|longitude|pattern_id|route_id|schedule_relationship|shift_id|speed|stop_id|          timestamp|             trip_id|           long_name| municipalities_name|      date|
+-------+--------+--------------+-------+---------+-------+---------+----------+--------+---------------------+--------+-----+-------+-------------------+--------------------+--------------------+--------------------+----------+
|      0|    1066|    STOPPED_AT|42|1066|38.972134|   2740|-9.419469|  2740_2_2|  2740_2|            SCHEDULED|   43745|  0.0| 080563|2024-12-01 23:17:57|2740_2_2|171|1|22...|Ericeira (Termina...|     [Mafra, Lisboa]|2024-12-01|
|      0|    1210|    STOPPED_AT|42|1210|38.816326|   2753|-9.154766|  2753_0_1|  27

In [12]:
analysis_df = (df
               .withColumn('municipalities_name', 
                           explode(df.municipalities_name)
                           )
               .groupBy('municipalities_name')
               .agg(
                   avg('speed').alias('average_speed'),
                   sum('speed').alias('sum_speed'),
                   count('id').alias('vehicle_count')
                   )
                )

In [20]:
print("What are the top 3 municipalities by vehicles routes?")
analysis_df.sort(desc('vehicle_count') ).show(3)
print("What are the top 3 municipalities with higher vehicle speed on average?")
analysis_df.sort(desc('average_speed') ).show(3)
analysis_df.sort(desc('sum_speed') ).show(3)

What are the top 3 municipalities by vehicles routes?
+-------------------+------------------+------------------+-------------+
|municipalities_name|     average_speed|         sum_speed|vehicle_count|
+-------------------+------------------+------------------+-------------+
|             Lisboa|  6.13675213777102| 398.8888889551163|           65|
|             Loures| 6.980286698187551| 216.3888876438141|           31|
|             Sintra|3.7547892660930238|108.88888871669769|           29|
+-------------------+------------------+------------------+-------------+
only showing top 3 rows

What are the top 3 municipalities with higher vehicle speed on average?
+-------------------+-----------------+-----------------+-------------+
|municipalities_name|    average_speed|        sum_speed|vehicle_count|
+-------------------+-----------------+-----------------+-------------+
|            Palmela| 8.41269862651825|58.88889038562775|            7|
|              Moita|8.148147900899252|48.8

# CHALLENGE 4
##  Analyze data

- Query table "vehicles_enriched" in gold layer
- Aggregate data by municipality_name (array)
- Calculate:
  - count of vehicles (id) that pass through that municipality
  - sum speed of vehicles

Questions:
  - What are the top 3 municipalities by vehicles routes?
  - What are the top 3 municipalities with higher vehicle speed on average?


Tips:
- explode array into rows -> https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/api/pyspark.sql.functions.explode.html
